In [1]:
#Test notebook for tracking rigid body  marker consisting of multiple fiducials within a CT scan volume

#3D data processing
import numpy as np
import open3d as o3d

#our processing code
from ct_tracking_library.ct_tracking_functions import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#choose target marker
target_marker = 'marker2'

#convert scan to mesh for finding markers via thresholding
convert_scan_to_mesh(scan_file='./test_data/bb_marker3.pkl')

#find candidate fiducials within the scanner returning clustered centroids that could be rigid body markers
marker, marker_centroid_coordinates, pcd_selected, mesh, coordinate_frame, good_centroid_clusters = find_candidate_centroids(target_marker = target_marker)

#display candidate fiducials
o3d.visualization.draw_geometries([pcd_selected, mesh])
print('marker centroid coordinates from design: {}'.format(marker))
print('found centroid coordinates: {}'.format(marker_centroid_coordinates))

#find best transform and marker from candidate fiducials
final_R, final_t, permuted_centroids, min_error = find_best_transform_from_candidate_marker_clusters(marker, good_centroid_clusters)

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 118


  0%|          | 0/118 [00:00<?, ?it/s]c:\Users\snowl\Documents\GitHub\ct-fiducial-tracking\ct_tracking_library\ct_tracking_functions.py:376: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  C, residules, rank, singval = np.linalg.lstsq(A,f)
100%|██████████| 118/118 [00:00<00:00, 538.26it/s]


[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 2
point cloud has 2 clusters
[0 1 2 3]
[4]
Multiple good clusters/centroids found
Multiple good clusters/centroids found
marker centroid coordinates from design: [[  0.           0.           0.        ]
 [-25.17330897   0.27807421  -0.31303004]
 [ -0.13682961  -1.03970283 -10.9369699 ]
 [-12.25998655  -1.57713288 -23.71379929]]
found centroid coordinates: [ 43.13876131  66.50936218 271.71275486]
Everything looks good!
the final error is:  0.12400797901918344


In [3]:
#display tracked  markers and coordinate frames
marker = np.load('./test_data/'+target_marker+'.npy')
marker_3d_base, marker_3d_transformed, err = visualize_tracked_marker(marker, final_R, final_t, permuted_centroids)
o3d.visualization.draw_geometries(marker_3d_base+marker_3d_transformed)

#tracking error from final transform
np.set_printoptions(2)
print(err)
print(np.linalg.norm(err))

#visualize everything at once
visualization_list = [pcd_selected, mesh]+marker_3d_base+marker_3d_transformed
o3d.visualization.draw_geometries(visualization_list)

[[-0.04 -0.01 -0.01]
 [ 0.13  0.04 -0.11]
 [-0.09  0.06  0.09]
 [-0.   -0.09  0.02]]
0.24801595803836687
